In [19]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [20]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[0.70577776, 0.62241596, 0.51488364, ..., 0.8772619 ,
         0.82996255, 0.77617264],
        [0.74917954, 0.6550527 , 0.546727  , ..., 0.9260574 ,
         0.8738347 , 0.8175615 ],
        [0.78415895, 0.68531847, 0.5583651 , ..., 0.96668965,
         0.913629  , 0.85359067],
        ...,
        [0.26230586, 0.26190177, 0.26149976, ..., 0.26352924,
         0.26311973, 0.26271188],
        [0.20862441, 0.20846592, 0.20830838, ..., 0.2091043 ,
         0.20894367, 0.20878367],
        [0.13749754, 0.13746928, 0.13744105, ..., 0.13758269,
         0.13755429, 0.1375259 ]],

       [[0.8827395 , 0.77422637, 0.6390555 , ..., 1.1195997 ,
         1.0507501 , 0.97485316],
        [0.9159353 , 0.7903128 , 0.64762056, ..., 1.1716399 ,
         1.0952063 , 1.0135571 ],
        [0.9547593 , 0.8281378 , 0.66257334, ..., 1.2162826 ,
         1.1372541 , 1.0501642 ],
        ...,
        [0.2524104 , 0.2520299 , 0.25165042, ..., 0.25355685,
         0.25317392, 0.25279173],
        [0.2

In [21]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [22]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4810,)


In [23]:
AO_ds.to_netcdf('EC-Earth3-AOindex-NDJF-Daily-1980-2014.nc')

In [24]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(3.5188515, dtype=float32)
Coordinates:
    time     datetime64[ns] 2006-02-10T12:00:00

In [25]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [26]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.5752363, dtype=float32)
Coordinates:
    time     datetime64[ns] 2005-12-25T12:00:00

In [27]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [28]:
zLevData = xr.open_dataset('../../../Data/CMIP6/EC-Earth3/processed/NDJF_New_zg_day_EC-Earth3_historical_r1i1p1f1_gr_18500101-20141231.nc')

In [29]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [30]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4810, lat: 256, lon: 512)>
[630456320 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * lat      (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-09-25T21:23:04Z altered by CMOR: Reordered dimension...

In [31]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [32]:
#z145E40N

In [33]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [34]:
#z20E55N

In [35]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [36]:
#z75E55N

In [37]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [38]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [39]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [40]:
EUVal

<xarray.DataArray 'zg' (time: 4810)>
array([ 0.11603606,  0.41104233,  0.44236464, ..., -0.63428581,
       -0.90969494, -1.40820576])
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 144.8

In [41]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [43]:
EU_ds.to_netcdf('EC-Earth3-EUindex-NDJF-Daily-1980-2014.nc')